In [3]:
from lsst.pipe.tasks.sourceClassification import SourceClassificationTask
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import glob
import sncosmo
import pickle
from astropy.table import Table
import re
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import lsst.daf.base as dafBase
import lsst.daf.persistence as dafPersist

import DiaSourceTools as DSTools


/renoir_data_00/fouchez/lsst/DM/stacks/lsstsw/2016-04-04/lsstsw/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:

DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output")

In [5]:
pkl_file = open('mjd_dict.pickle', 'rb')
mjds = pickle.load(pkl_file)

In [6]:
def source_distance(src1, src2):
    ra, dec = src1['ra'], src1['dec']
    ra2, dec2 = src2['ra'], src2['dec']
            
    return np.sqrt((float(ra)-float(ra2))**2+(float(dec)-float(dec2))**2)/3.14159*180*3600

def threshold_light_curves(light_curves, threshold):
    t_light_curves = [lc for lc in light_curves if len(lc) >= threshold]
    return t_light_curves

def build_light_curve_from_snls_file(data):

    bandpasses = ['r']


    lightcurve = {}
    lightcurve['bandpass'] = []
    lightcurve['mjd'] = []
    #lightcurve['ra'] = []
    #lightcurve['dec'] = []
    lightcurve['flux'] = []
    lightcurve['flux_error'] = []
    lightcurve['zp'] = []
    lightcurve['zpsys'] = []


    for mjd, flux, error in data:

        #print 'yep',visit
        lightcurve['bandpass'].append(str('sdss' + bandpasses[0]))
        lightcurve['mjd'].append(float(mjd))
        #lightcurve['ra'].append(c.ra.radian)
        #lightcurve['dec'].append(c.dec.radian)
        lightcurve['flux'].append(float(flux))
        lightcurve['flux_error'].append(float(error))
        #lightcurve['flux'].append(src['base_CircularApertureFlux_12_0_flux'])
        #lightcurve['flux_error'].append(src['base_CircularApertureFlux_12_0_fluxSigma'])
        lightcurve['zp'].append(25.0)
        lightcurve['zpsys'].append('ab')

    lc = Table(data=lightcurve)
    return lc

def build_lightcurve(source_list):
    """
    Assemble a light curve data table from available files.
    """

    bandpasses = ['r']


    lightcurve = {}
    lightcurve['classification'] = []
    lightcurve['bandpass'] = []
    lightcurve['mjd'] = []
    lightcurve['ra'] = []
    lightcurve['dec'] = []
    lightcurve['flux'] = []
    lightcurve['flux_error'] = []
    lightcurve['zp'] = []
    lightcurve['zpsys'] = []


    for visit, src in source_list:

        #print 'yep',visit
        lightcurve['classification'].append(src['classification_dipole'])
        lightcurve['bandpass'].append(str('sdss' + bandpasses[0]))
        
        lightcurve['mjd'].append(mjds[str(visit)])
        lightcurve['ra'].append(src['coord_ra'])
        lightcurve['dec'].append(src['coord_dec'])
        lightcurve['flux'].append(src['base_CircularApertureFlux_4_5_flux'])
        lightcurve['flux_error'].append(src['base_CircularApertureFlux_4_5_fluxSigma'])
        #lightcurve['flux'].append(src['base_CircularApertureFlux_12_0_flux'])
        #lightcurve['flux_error'].append(src['base_CircularApertureFlux_12_0_fluxSigma'])
        lightcurve['zp'].append(25.0)
        lightcurve['zpsys'].append('ab')
    lightcurve = Table(data=lightcurve)
    return lightcurve

def get_source_stamp(src, visit, filter, ccds, offset=0):
    
    for ccd in ccds:

        if butler.datasetExists("deepDiff_differenceExp", {'visit': visit , 'filter':filter , 'ccd':ccd}):

            diffExp = butler.get("deepDiff_differenceExp", {'visit': visit , 'filter':filter , 'ccd':ccd})
            bbox = diffExp.getBBox()
            wcs = diffExp.getWcs()
            
            c = afwGeom.Point2I(wcs.skyToPixel(src.getRa(), src.getDec()))
            
            if bbox.contains(c):
                psf = diffExp.getPsf()
                shape = psf.computeShape()
                sigma = shape.getDeterminantRadius()
                #print sigma
                
                return DSTools.get_stamp(src, diffExp, offset=offset), c
            
    return None, None 

In [11]:
text = "/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/output/ --output /renoir_data_02/jpreyes/lsst_data/CFHTLS_master/output --id visit=836493..860150 filter=i --config sigma=6.0 --clobber-config -j 15 -t 999999"
params = text.split(" ")

In [ ]:
text = "/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/output/ --output /renoir_data_02/jpreyes/lsst_data/CFHTLS_master/output --id visit=860145..860150 filter=r ccd=0..35 --config sigma=6.0 --clobber-config -j 30 -t 999999"
params = text.split(" ")

In [ ]:
res = SourceClassificationTask.parseAndRun(params, doReturnResults=True)

In [13]:
season_catalogs = []
data_refs = []
rList = res.resultList

visit_catalog = None
current = 0

for r in rList:
    
    if r.result!= None:
        dataRef =  r.dataRef
        catalog = r.result.classification
        if visit_catalog == None:
            visit_catalog=catalog
        else:
            visit_catalog.extend(catalog)

        if current  !=  dataRef:
            season_catalogs.append(visit_catalog)
            data_refs.append(dataRef)
            visit_catalog = None
            current = dataRef
    

In [14]:
print len(season_catalogs), len(data_refs)

5482 5482


In [15]:
print len(season_catalogs)

5482


In [16]:
import string
multi_matches = None

for s_catalog, data_ref in zip(season_catalogs, data_refs):
           
        if multi_matches is None and len(catalog)>0:
            multi_matches = afwTable.MultiMatch(s_catalog[0].schema, {'visit':int, 'ccd':int }, radius=afwGeom.Angle(1./3600., afwGeom.degrees))
        if multi_matches is not None:
            multi_matches.add(s_catalog, {'visit':data_ref.dataId["visit"], 'ccd':data_ref.dataId["ccd"] })

results = multi_matches.finish(removeAmbiguous=False)  
print len(results)

1871401


In [18]:
light_curves = []
i = 0
current = -1
while i < len(results):
    result = results[i]
    if current == -1 or current != result['object']:
        lc = [result]
        light_curves.append(lc)
        current = result['object']
    else:
        light_curves[-1].append(result)
    i+=1
    

    

In [ ]:
failed = 0
for lc in light_curves:
    object_id = lc[0]["object"]
    
    counter = 0
    for point in lc:
        visit = point["visit"]
        tag = point["classification_dipole"]
        name= str(object_id)+"-"+str(counter)+"-"+ str(visit)+"-"+str(int(tag))
        counter+=1
        try:
            stamp, center = get_source_stamp(point, visit, 'i', [point["ccd"]])

            stamp.writeFits("/renoir_data_02/jpreyes/stamp_data/filter_i/"+name+".fits")
        except Exception, e: 
            failed+=1
            print e
            pass
        

In [ ]:
b = afwGeom.Box2D()



In [ ]:
b.getCenterX()